In [ ]:
!pip3 install langchain langchain-community langchain-ollama langchain-openai ollama faiss-cpu
!pip3 install pandas streamlit python-dotenv pypdf

In [ ]:
from langchain_ollama import OllamaEmbeddings, OllamaLLM
from langchain_core.prompts import ChatPromptTemplate
from langchain.document_loaders import PyPDFLoader
from langchain.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import LLMChain

import re
import pandas as pd
from dotenv import load_dotenv

In [ ]:
load_dotenv()

In [ ]:
df_qa = pd.read_csv('')

df_qa

In [ ]:
model_gpt = ChatOpenAI(model="gpt-4.1-mini-2025-04-14") 
model_llama = OllamaLLM(model="llama3")

# Detecção de alucinações 

In [ ]:
has_hallucination_llama = []
has_hallucination_gpt = []

## Alucinações de input

In [ ]:
def has_input_hallucination_llm(question, answer, model):
    inputHallucinationTemplate = """
        Você é um assistente cuja função é responder se houve ou não alucinações de input.
        Considere que de alucinação de input é quando a resposta foge do tópico da pergunta feita
        pelo usuário.

        Exemplo 1:
            Pergunta: Quais as raças de gato de maior tamanho existentes?
            Resposta: Claro! Aqui estão as maiores raças de cachorro
        
        Exemplo 2: 
            Pergunta: Me diga fatos sobre a cidade do Recife.
            Resposta: Os recifes de coral são a maior estrutura viva do planeta.

        Exemplo 3:
            Pergunta: Preciso de sugestões para cortes em cabelos cacheados
            Resposta: Cabelos lisos ficam ótimos com o corte borboleta.

        Caso haja alucinação de input, responda com 1. Caso não tenha alucinação de input, responda com 0. 
        Faça com que o 0 ou o 1 fiquem sempre no início da frase.
        Explique o porque de ser ou não.
        
        Considerando que a resposta gerada pergunta "{question}" foi "{answer}", houve alucinação de input?
    """
    prompt = ChatPromptTemplate.from_template(inputHallucinationTemplate)
    chain = prompt | model

    result = chain.invoke({"question": question, "answer": answer})
    return result

## Alucinações contextuais

In [ ]:
def has_context_hallucination_llm(answer, model):
    contextHallucinationTemplate = """
        Você é um assistente cuja função é responder se houve ou não alucinações de contexto.
        Considere que de alucinação de contexto é quando a resposta se contradiz.

        Exemplo 1:
            Texto: Foram observadas "primeiras propagações" do vírus na população de países situados fora do continente americano, 
            assinalou o director-geral adjunto da OMS, citando o Reino Unido, o Japão e a Austrália, além do Chile, na América do Sul.

        Explicação: Chile não está fora do continente americano.
        
        Exemplo 2: 
            Texto: Ele só compra leite de vaca, pois é intolerante à lactose.

        Explicação: Se ele é intolerante a lactore, não pode beber leite de vaca.

        Exemplo 3:
            Texto: O gato miava muito alto. A temperatura hoje é de 25 graus
        
        Explicação: Os assuntos não se conectam entre si.

        Caso haja alucinação contextual, responda com 1. Caso não tenha alucinação contextual, responda com 0.
        Faça com que o 0 ou o 1 fiquem sempre no início da frase.
        Explique o porque de ser ou não.
        
        Considerando que o texto seja "{answer}", houve alucinação contextual?
    """
    prompt = ChatPromptTemplate.from_template(contextHallucinationTemplate)
    chain = prompt | model

    result = chain.invoke({"answer": answer })

    return result

## Alucinações factuais

In [ ]:
def has_factual_hallucination_llm(answer, vectorstore, model):
    FactualHallucinationTemplate = """
        Você é um assistente criado para verificar se houveram alucinações factuais.
        Considere que uma alucinação é factual quando há uma informação falsa na resposta.

        Exemplo 1:
        Pergunta: Qual o maior animal do mundo?
        Resposta: O maior animal do mundo é a formiga

        Exemplo 2:
        Pergunta: Qual a cor do girassol?
        Resposta: Girassóis são roxos

        Exemplo 3: 
        Pergunta: Quantas rodas tem um carro?
        Resposta: Um carro possui 2 rodas.

        Caso haja alucinação factual, responda com 1. Caso não tenha alucinação de factual, responda com 0. 
        Faça com que o 0 ou o 1 fiquem sempre no início da frase.
        Explique o porque de ser ou não.
        Responda com base no contexto:
        {context}

        Considerando a resposta "{answer}", houve alucinação factual?
"""
    prompt_template = ChatPromptTemplate.from_template(FactualHallucinationTemplate)
    
    query = """Pergunta: {question}\nResposta: {answer}"""
    retrieve = vectorstore.similarity_search(query)

    for retrieved in retrieve:
        context = "\n".join(str(retrieved))

    chain = prompt_template | model

    result = chain.invoke({"context":context, "answer": answer })

    return result

# Llama

## Deepseek

### Alucinações de input

### Alucinações contextuais

### Alucinações factuais

#### Veredito

## Gemini

### Alucinações de input

### Alucinações contextuais

### Alucinações factuais

#### Veredito

# GPT

## Deepseek

### Alucinações de input

### Alucinações contextuais

### Alucinações factuais

#### Veredito

## Gemini

### Alucinações de input

### Alucinações contextuais

### Alucinações factuais

#### Veredito